In [2]:
import torch
import os
import json
import glob
from collections import defaultdict
from os import path

In [4]:
# sample_file = "/home/shtoshni/1023_bleak_house_brat.conll.txt"
litbank_dir = "/home/shtoshni/Research/litbank_coref/litbank/coref/conll"
litbank_files = glob.glob("{}/*.conll".format(litbank_dir))

output_dir = "/home/shtoshni/Research/litbank_coref/litbank_html"
print(len(litbank_files))

100


In [12]:
def get_clusters(story_file):
    story_name = path.basename(story_file)
    with open(story_file) as f:
        count = 0
        sent_id = 0
        num_clusters = 0

        # Maintain list of all spans
        cluster_id_to_spans = defaultdict(list)

        # Maintain active clusters here
        cluster_id_to_active_spans = []
        
        for line_idx, line in enumerate(f.readlines()):
            line = line.strip()
            cols = line.split("\t")
            if len(cols) == 12:
                count += 1
                token = cols[3]
                # If there's an active cluster right now, process it
                cluster_id_to_active_spans = [
                    (cluster_id, active_span + " " + token)
                    for (cluster_id, active_span) in cluster_id_to_active_spans]

            elif len(cols) == 13:
                count += 1
                token = cols[3]
                cluster_token = cols[12]

                # Parse the cluster token
                all_clusters = cluster_token.split('|')
                ended_clusters = set([])
                
                cluster_id_to_active_spans = [
                    (cluster_id, active_span + " " + token)
                    for (cluster_id, active_span) in cluster_id_to_active_spans]
            
                for cluster_str in all_clusters:
                    if cluster_str[0] == '(' and cluster_str[-1] == ')':
                        cluster_idx = int(cluster_str[1:-1])
                        cluster_id_to_spans[cluster_idx].append(token)
                    elif cluster_str[0] == '(':
                        cluster_idx = int(cluster_str[1:])
                        cluster_id_to_active_spans.append((cluster_idx, token))
                    elif cluster_str[-1] == ')':
                        cluster_idx = int(cluster_str[:-1])
                        ended_clusters.add(cluster_idx)
                        for active_idx, (active_cluster_id, active_span) in enumerate(cluster_id_to_active_spans[::-1]):
                            if active_cluster_id == cluster_idx:
                                cluster_id_to_spans[cluster_idx].append(active_span)
                                break
    #                     print(cluster_id_to_active_spans[cluster_idx])
                        del cluster_id_to_active_spans[-(active_idx + 1)]
                    else:
                        print("Sweet Glory")
                        break
                
#                 for active_cluster_id, active_span in cluster_id_to_active_spans:
#                     if active_cluster_id in ended_clusters:
#                         print("Ambiguity")
#                         print("Cluster: {}, Line: {} Overlapping mention with the " 
#                               "same cluster id - {}".format(cluster_idx, line_idx, story_name))
# #                         print(cluster_id_to_active_spans)
                    

            if line == "":
                sent_id += 1
                
    return cluster_id_to_spans, count
            

In [98]:
total_mentions = 0
singleton_clusters = 0
total_tokens = 0

max_clusters = 0
max_cluster_story = None

story_to_cluster = {}
for story_file in litbank_files: 
    cluster_id_to_spans, token_cnt = get_clusters(story_file)
    story_to_cluster[story_file] = cluster_id_to_spans

    total_tokens += token_cnt
    max_clusters = max(max_clusters, len(cluster_id_to_spans))
    if max_clusters == len(cluster_id_to_spans):
        max_cluster_story = path.basename(story_file)
    
    for cluster_id in cluster_id_to_spans:
        cluster_mentions = len(cluster_id_to_spans[cluster_id])
        total_mentions += cluster_mentions
        if cluster_mentions == 1:
            singleton_clusters += 1

#         max_cluster_size = max(max_cluster_size, cluster_mentions)
#         if max_cluster_size == cluster_mentions:
#             max_cluster = cluster_id_to_spans[cluster_id]
    
print(singleton_clusters/total_mentions)
print(total_mentions)
print(total_tokens)
print("Max clusters ({}): {}".format(max_cluster_story, max_clusters))

# print("Max cluster size:", max_cluster_size)
# print("Max cluster:", max_cluster)



0.19791094007696536
29104
210532
Max clusters (940_the_last_of_the_mohicans_a_narrative_of_1757_brat.conll): 199


In [1]:
HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head><body>'

cluster_start_tag = '<div style="border:2px; display : inline; border-style:solid; padding: {}px; padding-right: 3px; padding-left: 3px">'
singleton_start_tag = '<div style="border:2px; display : inline; border-style:dotted; padding:{}px; padding-right: 3px; padding-left: 3px">'
end_tag = '</div>'

largest_padding = 11
padding_reduction = 3

overlap_cnt = 0


In [96]:

def return_html(story_file, cluster_id_to_spans):
    global overlap_cnt
    story_name = path.basename(story_file)
    html_string = HTML_START + '<div style="line-height: 3">'
    
    with open(story_file, encoding="utf-8") as f:
        count = 0
        sent_id = 0
        num_clusters = 0
            
        active_clusters = 0
        # Maintain active clusters here
        cluster_id_to_active_spans = []
        
        for line_idx, line in enumerate(f.readlines()):
            line = line.strip()
            cols = line.split("\t")
            if len(cols) == 12:
                count += 1
                token = cols[3]
                html_string += " " + token + " "
                # If there's an active cluster right now, process it
                cluster_id_to_active_spans = [
                    (cluster_id, active_span + " " + token)
                    for (cluster_id, active_span) in cluster_id_to_active_spans]

            elif len(cols) == 13:
                count += 1
                token = cols[3]
                cluster_token = cols[12]

                # Parse the cluster token
                all_clusters = cluster_token.split('|')
                ended_clusters = set([])
                
                multi_end = False
                
                cluster_id_to_active_spans = [
                    (cluster_id, active_span + " " + token)
                    for (cluster_id, active_span) in cluster_id_to_active_spans]
                
                # First add all the prefix borders i.e. boxes starting
                for idx, cluster_str in enumerate(all_clusters):
                    if cluster_str[0] == '(' and cluster_str[-1] != ')':
                        # Add prefix for multi-token mentions starting from current token
                        cluster_idx = int(cluster_str[1:])
                        prefix = cluster_start_tag
                        if len(cluster_id_to_spans[cluster_idx]) == 1:
                            prefix = singleton_start_tag
                        html_string += prefix.format(largest_padding - active_clusters * padding_reduction)
                        active_clusters += 1
                        cluster_id_to_active_spans.append((cluster_idx, token))
                
                for idx, cluster_str in enumerate(all_clusters):
                    if cluster_str[0] == '(' and cluster_str[-1] == ')':
                        # Single token clusters starting and ending at current token
                        cluster_idx = int(cluster_str[1:-1])
                        prefix = cluster_start_tag
                        if len(cluster_id_to_spans[cluster_idx]) == 1:
                            prefix = singleton_start_tag
                        
                        prefix_fmted = prefix.format(largest_padding - active_clusters * padding_reduction)
                        html_string += prefix_fmted
                    
                
                html_string += token + " "
                
                multiword_mentions = []
                for idx, cluster_str in enumerate(all_clusters):
                    if cluster_str[0] == '(' and cluster_str[-1] == ')':
                        # Single token clusters
                        cluster_idx = int(cluster_str[1:-1])
                        html_string += "<sub>" + str(cluster_idx) + "</sub>" + end_tag + " "
                    elif cluster_str[-1] == ')':
                        # Multi token clusters ending at current token
                        cluster_idx = int(cluster_str[:-1])
                        html_string += "<sub>" + str(cluster_idx) + "</sub>" + end_tag + " "
                        active_clusters -= 1 
                        multi_end = True
                        
                        for active_idx, (active_cluster_id, active_span) in enumerate(cluster_id_to_active_spans[::-1]):
                            if active_cluster_id == cluster_idx:
                                cluster_id_to_spans[cluster_idx].append(active_span)
                                break
    #                     print(cluster_id_to_active_spans[cluster_idx])
                        multiword_mentions.append(cluster_id_to_active_spans.pop(-(active_idx + 1)))
        
                if multi_end and active_clusters:
                    max_multi_end_length = max([len(s) for _, s in multiword_mentions])
                    min_multi_active_length = min([len(s) for _, s in cluster_id_to_active_spans])
                    
                    if max_multi_end_length >  min_multi_active_length:
                        # One of the spans that ended began before one of the current multiword spans.  
                        print("Overlap")
                        print("{} {} {} {} {}".format(story_name, sent_id, token, active_clusters, cluster_id_to_active_spans))
                        overlap_cnt += 1 
                    
            if line == "":
                sent_id += 1
                html_string += "<br/>\n" 
#                 break
    html_string += "</div></body></html>"
    return html_string

In [74]:
def extract_book_name(story_file):
    conll_file = path.basename(story_file)
    prefix = conll_file.split(".")[0]
    prefix_words = prefix.split("_")[1:-1]
    book_name = (" ".join(prefix_words)).capitalize()
    
    return book_name

In [97]:
index_html = HTML_START + '<ol type="1">'

for story_file in litbank_files:
    base_file = path.basename(story_file)
    output_file = base_file.replace("conll", "html")
    output_file = output_file.replace("_brat", "")
    
    book_name = extract_book_name(story_file)
    index_html += '<li> <a href="{}", target="_blank">'.format(output_file) + book_name + '</a></li>\n'
    
    book_html = return_html(story_file, story_to_cluster[story_file])
    with open(path.join(output_dir, output_file), "w") as f:
        f.write(book_html)
        

print(overlap_cnt)

index_html += '</ol>\n</body>\n</html>'
with open(path.join(output_dir, "index.html"), "w") as g:
    g.write(index_html)

Overlap
8867_the_magnificent_ambersons_brat.conll 39 man 1 [(99, 'man')]
Overlap
8867_the_magnificent_ambersons_brat.conll 39 youth 1 [(100, 'youth')]
Overlap
2891_howards_end_brat.conll 14 host 1 [(22, 'host')]
137
